source: https://iwatobipen.wordpress.com/2018/02/18/mol-encoder-with-pytorch/

In [1]:
import numpy as np
import torch
import argparse
from torch.autograd import Variable
from rdkit import Chem

In [2]:
import sys
sys.path.append('./molencoder/')

In [3]:
from molencoder.models import MolEncoder, MolDecoder
from molencoder.models import MolEncoder, MolDecoder
from molencoder.utils import( load_dataset, initialize_weights,ReduceLROnPlateau, save_checkpoint, validate_model)
from molencoder.featurizers import Featurizer, OneHotFeaturizer

In [4]:
SOURCE = 'c1ccccn1'
DEST =  'c1ccccc1'
STEPS = 200
#charset from chembl
WIDTH=120
 
charset = [' ', '#', '%', '(', ')', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '@', 'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'S', 'T', 'V', 'X', 'Z', '[', '\\', ']', 'a', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's', 't']
 

In [5]:
def decode_smiles_from_index(vec, charset):
    return "".join(map(lambda x:charset[x],vec)).strip()
 
 
def get_arguments():
    parser = argparse.ArgumentParser(description="Interpolate from source to dest in steps")
    parser.add_argument("--source", type=str, default=DEST)
    parser.add_argument("--dest", type=str, default=SOURCE)
    parser.add_argument("--steps", type=int, default=STEPS)
    return parser.parse_args()
 
def interpolate(source, dest, steps, charset, encoder, decoder):
    width=WIDTH
    source_just = source.ljust(width)
    dest_just = dest.ljust(width)
    onehot = OneHotFeaturizer(charset=charset)
    sourcevec = onehot.featurize(smiles=[source_just])
    destvec = onehot.featurize(smiles=[dest_just])
    source_encoded = Variable(torch.from_numpy(sourcevec).float()).cuda()
    dest_encoded = Variable(torch.from_numpy(destvec).float()).cuda()
    source_x_latent = encoder(source_encoded)
    dest_x_latent = encoder(dest_encoded)
    step = (dest_x_latent-source_x_latent)/float(steps)
    results = []
    for i in range(steps):
        item = source_x_latent + (step*i)
        sampled = np.argmax(decoder(item).cpu().data.numpy(), axis=2)
        #print(sampled)
        decode_smiles = decode_smiles_from_index(sampled[0], charset)
        results.append((i, item, decode_smiles))
    return results
 

In [9]:
#args= get_arguments()
encoder = MolEncoder( c = len(charset))
decoder = MolDecoder( c = len(charset))
encoder.apply(initialize_weights)
decoder.apply(initialize_weights)
 
print( torch.cuda.is_available() )
#encoder = MolEncoder( c = len(charset)).cuda()
encoder = MolEncoder( c = len(charset))
encoder.apply(initialize_weights)
 
#decoder = MolDecoder( c = len(charset)).cuda()
decoder = MolDecoder( c = len(charset))
decoder.apply(initialize_weights)
 
bestmodel = torch.load("model_best.pth.tar")
#bestmodel = torch.load("tempcheckpoint.pth.tar")
encoder.load_state_dict(bestmodel["encoder"])
decoder.load_state_dict(bestmodel["decoder"])

results = interpolate( args.source, args.dest, args.steps, charset, encoder, decoder )
for result in results:
    print(result[0], result[2])

True


FileNotFoundError: [Errno 2] No such file or directory: 'model_best.pth.tar'